<a href="https://colab.research.google.com/github/ka-means/Process-Mining-/blob/main/Process_Mining_with_pm4py_%E2%80%94_Traffic_Fines_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Mining Activity with pm4py on Traffic Fines Management

The goal is to practice Process Mining concepts by applying pm4py to a simulated event log of a traffic fines management process. Through log exploration and the Alpha algorithm, we answer multiple-choice and numeric questions. The work covers case counts, variants, start and end activities, frequency DFG, the footprint matrix, and discovery of a Petri net on filtered subsets of the log.

In [8]:
!pip -q install pm4py==2.7.7 pandas==2.2.2


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.7 MB/s eta 0:00:00


# Cargar y formatear el event log

In [14]:
import os, pandas as pd, pm4py
os.environ["CUDF_PANDAS_DISABLE"] = "1"

# Carga
df = pd.read_csv("/content/log_gestion_multas_transito.csv", sep=";")
df["Fin"] = pd.to_datetime(df["Fin"])

# Formatea a XES y crea event log
df = pm4py.format_dataframe(df, case_id="ID caso", activity_key="Actividad", timestamp_key="Fin")
df = df.rename(columns={"Ejecutor":"org:resource"})  # si existe, no pasa nada
elog = pm4py.convert_to_event_log(df)

# Helpers
def variants_of(elog_):
    from pm4py.statistics.traces.generic.log import case_statistics
    v = case_statistics.get_variant_statistics(elog_)
    return sorted(v, key=lambda x: x["count"], reverse=True)



### Chequeo

In [15]:
from pm4py.statistics.start_activities.log import get as sa
from pm4py.statistics.end_activities.log import get as ea

print("Casos:", len(elog))
print("Eventos:", sum(len(t) for t in elog))
print("Actividades:", len(pm4py.get_event_attribute_values(elog, "concept:name")))
print("Start:", sa.get_start_activities(elog))
print("End:", ea.get_end_activities(elog))


Casos: 25577
Eventos: 91082
Actividades: 8
Start: {'Crear multa': 25577}
End: {'Recibir pago': 17750, 'Enviar a recoleccion de credito': 5295, 'Apelacion multa': 711, 'Anadir penalizacion': 968, 'Insertar notificacion multa': 405, 'Insertar fecha apelacion multa': 8, 'Enviar multa': 14, 'Crear multa': 426}


# Preguntas  

Aplicar algoritmo Alpha:

*   Descubre la matriz de huella del proceso.
*   Obtén la red de Petri de una versión filtrada del event log.





### 1. ¿Cuántas ejecuciones del proceso (casos) tiene el event log?   

In [16]:
df['case:concept:name'].nunique()


25577

In [37]:
# 1) Verificar que todos los casos inician en "Crear multa"
df_sorted = df.sort_values(['case:concept:name','time:timestamp'])
starts = df_sorted.groupby('case:concept:name').first()['concept:name'].value_counts()
print(starts)  # debe mostrar solo "Crear multa" con 25577

# 2) Verificar que "Crear multa" aparece exactamente 1 vez por caso
cm_counts = df_sorted.assign(es_cm = df_sorted['concept:name'].eq('Crear multa')) \
                     .groupby('case:concept:name')['es_cm'].sum()
print("Mín:", cm_counts.min(), "Máx:", cm_counts.max())  # ambos deben ser 1
print("Casos con !=1 ocurrencias:", (cm_counts!=1).sum())  # debe ser 0


concept:name
Crear multa    25577
Name: count, dtype: Int64
Mín: 1 Máx: 1
Casos con !=1 ocurrencias: 0


### 2. ¿Cuántas actividades distintas tiene el proceso?  



In [17]:
df['concept:name'].nunique()


8

### 3. ¿Cuál es la actividad que más veces se realiza?  

In [18]:
df['concept:name'].value_counts().head(1)


,count
concept:name,
Crear multa,25577


### 4.  Sin contar al “Sistema”, ¿cuántos ejecutores tiene el proceso?

In [19]:
res_col = 'org:resource' if 'org:resource' in df.columns else 'Ejecutor'
df.loc[df[res_col].notna() & (df[res_col] != 'Sistema'), res_col].nunique()


18

### 5.  ¿Cuál es el tiempo promedio que tarda el proceso en ejecutarse?  

In [20]:
tmp = df.sort_values(['case:concept:name','time:timestamp']).groupby('case:concept:name').agg(
    start=('time:timestamp','first'), end=('time:timestamp','last'))
secs = (tmp['end'] - tmp['start']).dt.total_seconds()
secs.mean(), secs.mean()/86400


(np.float64(10449181.170934824), np.float64(120.93959688581973))

### 6. ¿Cuál de las siguientes secuencias de actividades corresponde a la variante más común?  

In [21]:
vars_all = variants_of(elog)
vars_all[0]['variant'], vars_all[0]['count']


(('Crear multa', 'Recibir pago'), 10999)

### 7. Filtra el proceso para dejar únicamente los casos terminados en ‘Recibir pago’ y ‘Enviar a recolección de crédito’. ¿Cuántas variantes (formas de ejecutar el proceso) quedan?

In [27]:
# 1) calcular los casos a conservar por actividad final
ends = {t.attributes["concept:name"]: t[-1]["concept:name"] for t in elog}
keep_ids = [cid for cid,a in ends.items() if a in ["Recibir pago","Enviar a recoleccion de credito"]]

# 2) filtrar el DataFrame por esos case ids
df_f = df[df['case:concept:name'].isin(keep_ids)]

# 3) reconvertir a EventLog
elog_f = pm4py.convert_to_event_log(df_f)

# 4) contar variantes
from pm4py.statistics.traces.generic.log import case_statistics
vars_f = case_statistics.get_variant_statistics(elog_f)
len(vars_f)


9

### 8. Al revisar la matriz de huellas del log de eventos, ¿qué par de actividades se ejecutan en paralelismo?

In [23]:
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
dfg = dfg_discovery.apply(elog, variant=dfg_discovery.Variants.FREQUENCY)
def both(a,b): return ((a,b) in dfg) and ((b,a) in dfg)
pairs = {
    ("Anadir penalizacion","Recibir pago"): both("Anadir penalizacion","Recibir pago"),
    ("Anadir penalizacion","Apelacion multa"): both("Anadir penalizacion","Apelacion multa"),
    ("Apelacion multa","Enviar a recoleccion de credito"): both("Apelacion multa","Enviar a recoleccion de credito"),
    ("Enviar multa","Insertar notificacion multa"): both("Enviar multa","Insertar notificacion multa"),
}
pairs


{('Anadir penalizacion', 'Recibir pago'): False,
 ('Anadir penalizacion', 'Apelacion multa'): True,
 ('Apelacion multa', 'Enviar a recoleccion de credito'): False,
 ('Enviar multa', 'Insertar notificacion multa'): False}

### 9. Filtra el proceso para dejar solo los casos terminados en ‘Enviar a recolección de crédito’. ¿Cuál de las siguientes secuencias de actividades corresponde a la variante más común entre las que quedaron?

In [30]:
# 1) obtener los casos cuyo último evento es 'Enviar a recoleccion de credito'
df_sorted = df.sort_values(['case:concept:name','time:timestamp'])
ends_df = df_sorted.groupby('case:concept:name').tail(1)
keep_ids = ends_df.loc[
    ends_df['concept:name'] == 'Enviar a recoleccion de credito',
    'case:concept:name'
].unique().tolist()

# 2) filtrar el DataFrame y reconvertir a EventLog
df_c = df[df['case:concept:name'].isin(keep_ids)]
elog_c = pm4py.convert_to_event_log(df_c)

# 3) contar variantes y tomar la más común
from pm4py.statistics.traces.generic.log import case_statistics
vars_c = case_statistics.get_variant_statistics(elog_c)
vars_c = sorted(vars_c, key=lambda x: x['count'], reverse=True)
vars_c[0]['variant'], vars_c[0]['count']




(('Crear multa',
  'Enviar multa',
  'Insertar notificacion multa',
  'Anadir penalizacion',
  'Enviar a recoleccion de credito'),
 5000)

In [32]:
v = vars_c[0]['variant']
if isinstance(v, (list, tuple)):
    v_str = ", ".join(v)
else:
    v_str = str(v).replace(",", ", ")
print(v_str, " · casos:", vars_c[0]['count'])



Crear multa, Enviar multa, Insertar notificacion multa, Anadir penalizacion, Enviar a recoleccion de credito  · casos: 5000


### 10. Filtra el proceso para seleccionar solo los casos en donde la Severidad de la multa es igual a 3.0 ¿Cuántas ejecuciones del proceso (casos) quedan?  



In [26]:
df.loc[df['Severidad']==3.0, 'case:concept:name'].nunique()


4661

### 11. Filtra el proceso para dejar solo los casos terminados en ‘Recibir pago’ y ‘Enviar a recolección de crédito’. ¿Cuántos places y transiciones se generan al ejecutar el algoritmo Alpha sobre el log filtrado?  


In [35]:
import pm4py
import pandas as pd


# identificar los casos cuyo último evento es pago o recolección
df_sorted = df.sort_values(['case:concept:name','time:timestamp'])
ends_df = df_sorted.groupby('case:concept:name').tail(1)
keep_ids = ends_df.loc[
    ends_df['concept:name'].isin(['Recibir pago','Enviar a recoleccion de credito']),
    'case:concept:name'
].unique().tolist()

# filtrar el DataFrame y convertir a EventLog
df_f = df[df['case:concept:name'].isin(keep_ids)]
elog_f = pm4py.convert_to_event_log(df_f)

# chequeo rápido
from pm4py.statistics.traces.generic.log import case_statistics
print("Casos filtrados:", len(elog_f))
print("Variantes filtradas:", len(case_statistics.get_variant_statistics(elog_f)))



Casos filtrados: 23045
Variantes filtradas: 9


In [36]:
from pm4py.algo.discovery.alpha import algorithm as alpha_miner

net, im, fm = alpha_miner.apply(elog_f)
print("Places:", len(net.places))
print("Transiciones:", len(net.transitions))

# si quieres ver los nombres de las transiciones
print(sorted([t.label for t in net.transitions if t.label is not None]))


Places: 12
Transiciones: 8
['Anadir penalizacion', 'Apelacion multa', 'Crear multa', 'Enviar a recoleccion de credito', 'Enviar multa', 'Insertar fecha apelacion multa', 'Insertar notificacion multa', 'Recibir pago']
